Go to https://datacentral.org.au/services/query/
![galah query page](<images/galah-query-page.png>)

The query
```
SELECT TOP 100000 galah_main.rv_galah,
galah_main.fe_h AS feh_galah, gaia_vac.dr3_source_id AS source_id, 
galah_main.alpha_fe AS alphafe_galah, galah_main.teff AS teff_galah,
galah_main.e_teff AS e_teff_galah, galah_main.logg AS logg_galah,
galah_main.e_logg AS e_logg_galah
FROM galah_dr3p2.main_star as galah_main
INNER JOIN galah_dr3p2.vac_gaia_edr3 as gaia_vac on galah_main.dr3_source_id = gaia_vac.dr3_source_id
WHERE galah_main.flag_sp = 0 AND galah_main.flag_fe_h = 0
AND galah_main.flag_alpha_fe = 0
AND galah_main.ra_dr2 BETWEEN 60 AND 120
```
Divide RA in smaller chunk if necessary

Data downloaded have `byte` data type, which is annoying.

convert it to `float64`

import vaex
from glob import glob
from astropy.table import Table

In [ ]:
# load downloaded data in this directory with .fits format

files = glob("*.fits")
files.sort()

In [ ]:
# convert the data types

df_com = []
for file in files:
    table = Table.read(file)
    for col in table.colnames:
        table[col] = table[col].astype("float64")
    df = vaex.from_astropy_table(table)
    if len(df_com) == 0:
        df_com = df
    else:
        df_com = df_com.concat(df)

In [ ]:
# export to hdf5

df_com.export("galah.hdf5", progress=True)